In [35]:
from app import *

sheets = extract_sheets()
market = fetch_market(sheets['Dicts']['assets'], sheets['Others'])
forex = fetch_forex(sheets['Dicts']['depots'])
data = build_metrics(sheets['Dicts']['assets'], market, sheets['Dicts']['depots'], forex, sheets['Operation'])



[*********************100%***********************]  3 of 3 completed
[*********************100%***********************]  2 of 2 completed
 DMA
   €
 ZEN
   $
   BTC
   ESE
   GOLD
   GREENBULL
   USDT
   €


In [64]:
df = data[['ValueEUR', 'InvestedEUR', 'CashEUR', 'PnLEUR', 'DepositEUR']]
df = pd.concat([df['ValueEUR', 'ZEN', 'All'], df['ValueEUR', 'DMA', 'All'], df['CashEUR', 'All', 'All']], axis=1)
df.columns = ['ZEN', 'DMA', 'Cash'] 
df.iloc[-1].index

Index(['ZEN', 'DMA', 'Cash'], dtype='object')

In [70]:
df = data[['ValueEUR', 'InvestedEUR', 'CashEUR', 'PnLEUR', 'DepositEUR']]
df_pie = pd.concat({ 
    'ZEN' : df['ValueEUR', 'ZEN', 'All'],  
    'DMA' : df['ValueEUR', 'DMA', 'All'],  
    'Cash' : df['CashEUR', 'All', 'All']
}, axis=1) 

df_pie.columns

Index(['ZEN', 'DMA', 'Cash'], dtype='object')

In [48]:
print(s_last['CashEUR', 'All', 'All'])
s_pie = s_last['ValueEUR', :, 'All'].drop(labels=['All']) #.append(s_last['CashEUR', 'All', 'All'])
s_pie

11125.33017654419


DMA       0.000000
ZEN    8452.909579
Name: 2023-01-12 00:00:00, dtype: float64